In [11]:
# Load pickle file
import pickle
import pandas as pd
import numpy as np
import scipy.linalg as la
import cvxpy as cp
import altair as alt
from statsmodels.stats.correlation_tools import cov_nearest
from numpy.linalg import solve, norm

In [12]:
# Functions
def log_return(df):
    return np.log(df).diff().drop(index=df.index[0], axis=0, inplace=False)

def create_index(df):
    # the linear method already does ffill for the tail values
    return na_fill(df.mean(axis=1, skipna=False))

def na_fill(df):
    # Simulates `na.fill(as.xts(rowAvgs(df),order.by=index(df)),"extend")`
    return df.interpolate(method="linear").fillna(method="bfill")

# Load data

In [13]:
# Stocks
dataset= pd.read_pickle('data/dataset.pkl')

# FFM
fama_lib = pd.read_csv("data/F-F_Research_Data_Factors_daily.CSV", skiprows=4, skipfooter=1, index_col=0, engine='python')
fama_lib.index = pd.to_datetime(fama_lib.index, format="%Y%m%d")
fama_lib = fama_lib.iloc[:, 0:3] # We only use the first 3 columns

In [14]:
# Prepare stock data
begin_date = "2016-01-01"  
end_date = "2018-01-01" 
stockPrices = dataset["adjusted"].truncate(
    before=begin_date,
    after=end_date
)

# Keep only the stocks that are present in the FFM dataset
stocks = ["GOOG", "HAL", "HSBC", "JPM", "KO", "MCD", "MSFT","PFE", "XOM"]
stockPrices = stockPrices.loc[:, stocks]

In [15]:
# Compute log returns
X = log_return(stockPrices)
T, N = X.shape  # nrow and ncol

# Compute factor models
F_FamaFrench = fama_lib.loc[X.index]/100   # Fama-French factors
BBrMkt = create_index(dataset["BBr"])/100  # BBr Market index
PNlogMkt = create_index(dataset["PNlog"])  # PNlog Market index
SentIndx = PNlogMkt.loc[X.index]           # Sentiment index

/var/folders/js/pnl2ymz143j6txn5bftwltm40000gn/T/ipykernel_15792/2546399395.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.interpolate(method="linear").fillna(method="bfill")
/var/folders/js/pnl2ymz143j6txn5bftwltm40000gn/T/ipykernel_15792/2546399395.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.interpolate(method="linear").fillna(method="bfill")


# Robust (ellipsoid) Global Maximum Return Portfolio

In [16]:
def portfolioMaxReturnRobustEllipsoid(mu_hat, S, kappa=0.1):
    mu_hat = np.array(mu_hat)
    S12 = la.cholesky(S)  # S12.T @ S12 = Sigma
    w = cp.Variable(len(mu_hat))
    prob = cp.Problem(cp.Maximize(mu_hat @ w - kappa * cp.norm(S12 @ w, p=2)), constraints=[w >= 0, sum(w) == 1])
    result = prob.solve()
    return w.value

def is_pos_def(x, epsilon):
    return np.all(np.linalg.eigvals(x) > epsilon)

## Using FFM

In [17]:
def calculate_robust_portfolio_weights_ffm(stocks, mu, Sigma, T, kappa):

    np.random.seed(110)

    # Initialize the array to store weights
    w_all_GMRP_robust_ellipsoid = np.empty((len(stocks), 0))
    
    for i in range(6):
        X_noisy = pd.DataFrame(np.random.multivariate_normal(mu, Sigma, size=T))
        mu_noisy = X_noisy.mean(axis=0)

        ################
        # Calculate Sigma using FFM
        ################
        F = F_FamaFrench.copy()
        F.insert(0, 'ones', 1)
        Gamma = pd.DataFrame(solve(F.T @ F, F.T.to_numpy() @ X_noisy.to_numpy()).T, columns=["alpha", "beta1", "beta2", "beta3"])
        alpha = Gamma["alpha"]
        B = Gamma[["beta1", "beta2", "beta3"]]
        E = pd.DataFrame((X_noisy.T - Gamma.to_numpy() @ F.to_numpy().T).T, index=X_noisy.index)
        PsiFF = (E.T @ E) / (T - 4)
        Sigma_FamaFrench = B.to_numpy() @ F_FamaFrench.cov() @ B.to_numpy().T + np.diag(np.diag(PsiFF))

        Sigma_noisy = Sigma_FamaFrench

        # Ensure positive definiteness of the covariance matrix
        if not is_pos_def(Sigma_noisy, 0.001):
            Sigma_noisy = cov_nearest(Sigma_noisy)

        w_GMRP_robust_ellipsoid_noisy = portfolioMaxReturnRobustEllipsoid(mu_noisy, Sigma_noisy, kappa)
        w_all_GMRP_robust_ellipsoid = np.concatenate(
            (w_all_GMRP_robust_ellipsoid, np.expand_dims(w_GMRP_robust_ellipsoid_noisy, axis=1)), axis=1)

    # Convert weights to DataFrame and add stock names
    w_all_GMRP_robust_ellipsoid = pd.DataFrame(w_all_GMRP_robust_ellipsoid)
    w_all_GMRP_robust_ellipsoid["stock"] = stocks

    # Prepare data for visualization
    long_w_all_GMRP_robust_ellipsoid = pd.melt(
        w_all_GMRP_robust_ellipsoid, id_vars=['stock'], var_name='run', value_name='weight')

    # Create and return the chart
    chart = alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar(size=10, color='red').encode(
        x=alt.X('run:O', axis=None),
         y=alt.Y('weight:Q', scale=alt.Scale(domain=[0, 0.3])),
        column=alt.Column('stock:N', spacing=2)
    ).properties(width=125, title=f'Portfolio weight by stock and noisy run (kappa={kappa})')
    
    return chart

In [18]:
mu = X.mean(axis=0)
Sigma = X.cov()

for kappa in [0.25, 0.5, 75]:
    chart = calculate_robust_portfolio_weights_ffm(stocks, mu, Sigma, T, kappa)
    chart.save(f'figures/robust_portfolio_weights_ffm_kappa_{kappa}.png')
    chart.show()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Using PNlog factor model

In [21]:
def calculate_robust_portfolio_weights_PNlog(stocks, mu, Sigma, T, kappa):

    np.random.seed(110)

    # Initialize the array to store weights
    w_all_GMRP_robust_ellipsoid = np.empty((len(stocks), 0))
    
    for i in range(6):
        X_noisy = pd.DataFrame(np.random.multivariate_normal(mu, Sigma, size=T))
        mu_noisy = X_noisy.mean(axis=0)

        ################
        # Calculate sigma using PNlog
        ################
        F = SentIndx.copy().to_frame()
        F.insert(0, 'ones', 1)
        Gamma = pd.DataFrame(solve(F.T @ F, F.T.to_numpy() @ X.to_numpy()).T, columns=["alpha", "beta"])
        alpha = Gamma["alpha"]
        beta = Gamma["beta"]
        E = pd.DataFrame((X.T - Gamma.to_numpy() @ F.to_numpy().T).T, index=X.index)
        Psi_Sent = (E.T @ E) / (T - 2)
        Sigma_SentInx = SentIndx.var() * np.outer(beta, beta) + np.diag(np.diag(Psi_Sent))

        Sigma_noisy = Sigma_SentInx

        # Ensure positive definiteness of the covariance matrix
        if not is_pos_def(Sigma_noisy, 0.001):
            Sigma_noisy = cov_nearest(Sigma_noisy)

        w_GMRP_robust_ellipsoid_noisy = portfolioMaxReturnRobustEllipsoid(mu_noisy, Sigma_noisy, kappa)
        w_all_GMRP_robust_ellipsoid = np.concatenate(
            (w_all_GMRP_robust_ellipsoid, np.expand_dims(w_GMRP_robust_ellipsoid_noisy, axis=1)), axis=1)

    # Convert weights to DataFrame and add stock names
    w_all_GMRP_robust_ellipsoid = pd.DataFrame(w_all_GMRP_robust_ellipsoid)
    w_all_GMRP_robust_ellipsoid["stock"] = stocks

    # Prepare data for visualization
    long_w_all_GMRP_robust_ellipsoid = pd.melt(
        w_all_GMRP_robust_ellipsoid, id_vars=['stock'], var_name='run', value_name='weight')

    # Create and return the chart
    chart = alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar(size=10, color='red').encode(
        x=alt.X('run:O', axis=None),
         y=alt.Y('weight:Q', scale=alt.Scale(domain=[0, 0.3])),
        column=alt.Column('stock:N', spacing=2)
    ).properties(width=125, title=f'Portfolio weight by stock and noisy run (kappa={kappa})')
    
    return chart

In [22]:
mu = X.mean(axis=0)
Sigma = X.cov()

for kappa in [0.25, 0.5, 75]:
    chart = calculate_robust_portfolio_weights_PNlog(stocks, mu, Sigma, T, kappa)
    chart.save(f'figures/robust_portfolio_weights_pnglog_kappa_{kappa}.png')
    chart.show()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)